# Sentence Classification with BERT

Turn a pre-trained BERT model into a trainable Keras layer and apply it to the semeval 2017. BERT (Bidirectional Embedding Representations from Transformers) is a new model for pre-training language representations that obtains state-of-the-art results on many NLP tasks. We demonstrate how to integrate BERT as a custom Keras layer to simplify model prototyping using huggingface. 


In [1]:
import keras
import numpy as np
from keras.layers import Lambda, GlobalAveragePooling1D, Dense, Embedding
from keras import backend as K
from keras.models import Sequential
import matplotlib.pyplot as plt

from keras.layers import LSTM, RNN, Dropout, Input, LeakyReLU, Bidirectional,Conv1D, GlobalMaxPooling1D
from keras.layers.core import Dense
from keras.models import Model
from keras.utils import plot_model


Before we start, we should install the huggingface transformer package. You can find the doc from its [website](https://huggingface.co/transformers/index.html).

In [3]:
pip install transformers

     |████████████████████████████████| 2.2MB 6.1MB/s 
     |████████████████████████████████| 3.3MB 23.4MB/s 
     |████████████████████████████████| 870kB 39.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=f9f92b5b5471d971fdc9fd63fb330bb131e38acfa143c28126fcbdb304d2ce83
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


## Preprocessing and Tokenization

In this lab we will use DistilBERT instead of BERT: DistilBERT is a small, fast, cheap and light Transformer model trained by distilling BERT base. It has 40% less parameters than bert-base-uncased, and runs 60% faster, while preserving over 95% of BERT’s performance as measured on the GLUE language understanding benchmark.

It is easy to switch between DistilBERT and BERT using the huggingface transformer package. This huggingface package provides many pre-trained and pre-built models that are easy to use via a few lines of code.

Before using DistilBERT or BERT, we need a tokenizer. Generally speaking, every BERT related model has its own tokenizer, trained for that model (see this week's lecture video on sub-word tokenization). 
We can get the DistilBERT tokenizer from **DistilBertTokenizer.from_pretrained** function.

In [4]:
from transformers import DistilBertTokenizer, RobertaTokenizer 
import tqdm
distil_bert = 'distilbert-base-uncased' # Pick any desired pre-trained model

# Defining DistilBERT tokonizer
tokenizer = DistilBertTokenizer.from_pretrained(distil_bert, do_lower_case=True, add_special_tokens=True,
                                                max_length=128, pad_to_max_length=True)

def tokenize(sentences, tokenizer, pad_length=128, pad_to_max_length=True ):
    if type(sentences) == str:
        inputs = tokenizer.encode_plus(sentences, add_special_tokens=True, max_length=pad_length, pad_to_max_length=pad_to_max_length, 
                                             return_attention_mask=True, return_token_type_ids=True)
        return np.asarray(inputs['input_ids'], dtype='int32'), np.asarray(inputs['attention_mask'], dtype='int32'), np.asarray(inputs['token_type_ids'], dtype='int32')
    input_ids, input_masks, input_segments = [],[],[]
    for sentence in sentences:
        inputs = tokenizer.encode_plus(sentence, add_special_tokens=True, max_length=pad_length, pad_to_max_length=pad_to_max_length, 
                                             return_attention_mask=True, return_token_type_ids=True)
        input_ids.append(inputs['input_ids'])
        input_masks.append(inputs['attention_mask'])
        input_segments.append(inputs['token_type_ids'])        
        
    return np.asarray(input_ids, dtype='int32'), np.asarray(input_masks, dtype='int32'), np.asarray(input_segments, dtype='int32')


Then we can use the tokenizer to tokenize the sentence. When working with word2vec and GloVe, we tokenized the sentence into words ourselves and then converted the tokens to GloVe word indices. But in BERT, we must use the BERT tokenizer: the tokens for BERT are different, and include whole words and sub-word tokens (see lecture video on sub-word tokenisation).

For example, for the sentence: **This is a pretrained model.** our previous word-based tokenizer will generate the following tokens:

**"this", "is", "a", "pretrained", "model", "."**

Then you will find out that the word token "pretrained" is not in the GloVe word dictionary. Thus we can not assign a proper word vector for "pretrained".

In BERT, the BERT tokenizer will separate the word "pretrained" into three sub-word tokens:

**'pre', '##train', '##ed'**

This way, BERT can use these three token vectors to represent the word "pretrained". Without the BERT tokenizer, it is hard to separate these unknown words properly.

In [5]:
inputs = tokenizer.tokenize("The capital of France is [MASK].")
print(inputs,'\n')

inputs = tokenizer.tokenize("This is a pretrained model.")
print(inputs,'\n')

ids,masks,segments = tokenize("The capital of France is [MASK].", tokenizer)
print(ids)
print(masks,"\n")

ids,masks,segments = tokenize("The capital of France is [MASK].", tokenizer, pad_to_max_length=False)
print(ids)
print(masks)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


['the', 'capital', 'of', 'france', 'is', '[MASK]', '.'] 

['this', 'is', 'a', 'pre', '##train', '##ed', 'model', '.'] 

[ 101 1996 3007 1997 2605 2003  103 1012  102    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
[1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


## Loading Data

### Downloading and preprocessing data

In [6]:
import requests
def downloadfile(url):
  rq = requests.get(url)
  open(url.split('/')[-1], 'wb').write(rq.content)

downloadfile('https://raw.githubusercontent.com/cbaziotis/datastories-semeval2017-task4/master/dataset/Subtask_BD/downloaded/twitter-2016devtest-BD.tsv')
downloadfile('https://raw.githubusercontent.com/cbaziotis/datastories-semeval2017-task4/master/dataset/Subtask_BD/downloaded/twitter-2016dev-BD.tsv')
downloadfile('https://raw.githubusercontent.com/cbaziotis/datastories-semeval2017-task4/master/dataset/Subtask_BD/downloaded/twitter-2016test-BD.tsv')
downloadfile('https://raw.githubusercontent.com/cbaziotis/datastories-semeval2017-task4/master/dataset/Subtask_BD/downloaded/twitter-2016train-BD.tsv')
downloadfile('https://raw.githubusercontent.com/cbaziotis/datastories-semeval2017-task4/master/dataset/Subtask_BD/downloaded/twitter-2015test-BD.tsv')
downloadfile('https://raw.githubusercontent.com/cbaziotis/datastories-semeval2017-task4/master/dataset/Subtask_BD/downloaded/twitter-2015train-BD.tsv')

downloadfile('https://raw.githubusercontent.com/cbaziotis/datastories-semeval2017-task4/master/dataset/Subtask_BD/gold/SemEval2017-task4-test.subtask-BD.english.txt')

with open('twitter-2016dev-BD.tsv', 'r',encoding='utf-8') as f:
  dev_original = [l.strip().split('\t') for l in f.readlines()]
with open('SemEval2017-task4-test.subtask-BD.english.txt', 'r',encoding='utf-8') as f:
  test_original = [l.strip().split('\t') for l in f.readlines()]
train_original = []
with open('twitter-2016train-BD.tsv', 'r',encoding='utf-8') as f:
  train_original = [l.strip().split('\t') for l in f.readlines()]
with open('twitter-2016test-BD.tsv', 'r',encoding='utf-8') as f:
  train_original.extend([l.strip().split('\t') for l in f.readlines()])
with open('twitter-2016devtest-BD.tsv', 'r',encoding='utf-8') as f:
  train_original.extend([l.strip().split('\t') for l in f.readlines()])
with open('twitter-2015test-BD.tsv', 'r',encoding='utf-8') as f:
  train_original.extend([l.strip().split('\t') for l in f.readlines() if l.strip().split('\t')[2] in ['negative','positive']])
with open('twitter-2015train-BD.tsv', 'r',encoding='utf-8') as f:
  train_original.extend([l.strip().split('\t') for l in f.readlines() if l.strip().split('\t')[2] in ['negative','positive']])

print("Training entries: {}".format(len(train_original)))
print("Development entries: {}".format(len(dev_original)))
print("Testing entries: {}".format(len(test_original)))

Training entries: 17639
Development entries: 1325
Testing entries: 6185


In [7]:
print("ID \t TOPIC \t LABLE \t TWEET_TEXT")
print(train_original[0])
print(train_original[1])
print(train_original[2])
print(train_original[3])
print(train_original[4])

ID 	 TOPIC 	 LABLE 	 TWEET_TEXT
['628949369883000832', '@microsoft', 'negative', "dear @Microsoft the newOoffice for Mac is great and all, but no Lync update? C'mon."]
['628976607420645377', '@microsoft', 'negative', "@Microsoft how about you make a system that doesn't eat my friggin discs. This is the 2nd time this has happened and I am so sick of it!"]
['629023169169518592', '@microsoft', 'negative', "I may be ignorant on this issue but... should we celebrate @Microsoft's parental leave changes? Doesn't the gender divide suggest... (1/2)"]
['629179223232479232', '@microsoft', 'negative', 'Thanks to @microsoft, I just may be switching over to @apple.']
['629226490152914944', '@microsoft', 'positive', 'Microsoft, I may not prefer your gaming branch of business. But, you do make a damn fine operating system. #Windows10 @Microsoft']


In [8]:
# Please write your code to generate the following data
#x_train_tweet_int
#x_train_tweet_masks
# x_train_topic_int
# x_train_topic_masks

# x_dev_tweet_int
# x_dev_tweet_masks
# x_dev_topic_int
# x_dev_topic_masks

# x_test_tweet_int
# x_test_tweet_masks
# x_test_topic_int
# x_test_topic_masks

x_train_tweetb4tokenize = []
x_train_topicb4tokenize = []
x_dev_tweetb4tokenize = []
x_dev_topicb4tokenize = []
x_test_tweetb4tokenize = []
x_test_topicb4tokenize = []


# your code goes here

for w in train_original:                        #Extracts the tweet and topics from train set
    x_train_tweetb4tokenize.append(w[3])
    x_train_topicb4tokenize.append(w[1])

for w in dev_original:                          #Extracts the tweet and topics from dev set
    x_dev_tweetb4tokenize.append(w[3])
    x_dev_topicb4tokenize.append(w[1])

for w in test_original:                         #Extracts the tweet and topics from test set
    x_test_tweetb4tokenize.append(w[3])
    x_test_topicb4tokenize.append(w[1])

x_train_tweet_int,x_train_tweet_masks,x_train_tweet_segments = tokenize(x_train_tweetb4tokenize, tokenizer)  
x_train_topic_int,x_train_topic_masks,x_train_topic_segments = tokenize(x_train_topicb4tokenize, tokenizer)

x_dev_tweet_int,x_dev_tweet_masks,x_dev_tweet_segments = tokenize(x_dev_tweetb4tokenize, tokenizer)
x_dev_topic_int,x_dev_topic_masks,x_dev_topic_segments = tokenize(x_dev_topicb4tokenize, tokenizer)

x_test_tweet_int,x_test_tweet_masks,x_test_tweet_segments = tokenize(x_test_tweetb4tokenize, tokenizer)  
x_test_topic_int,x_test_topic_masks,x_test_topic_segments = tokenize(x_test_topicb4tokenize, tokenizer)

# If use the previous tokenize function, you can get a print result like:
assert len(x_train_topic_int) == len(train_original)
assert len(x_train_topic_masks) == len(x_train_topic_int)
assert len(x_test_topic_int) == len(test_original)
assert len(x_test_topic_masks) == len(x_test_topic_int)
print("x_dev_topic_int[0]:")
print(x_dev_topic_int[0])
print("x_dev_topic_masks[0]:")
print(x_dev_topic_masks[0])
print("x_dev_tweet_int[0]:")
print(x_dev_tweet_int[0])
print("x_dev_tweet_masks[0]:")
print(x_dev_tweet_masks[0])


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


x_dev_topic_int[0]:
[ 101 2745 4027  102    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
x_dev_topic_masks[0]:
[1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
x_dev_tweet_int[0]:
[  101  6108  1062  9794 1

We use 1 to represent "positive" and 0 for "negative" and generate the "y" data.

In [9]:
def label2int(dataset):
  y = []
  for example in dataset:
    if example[2].lower() == "negative":
      y.append(0)
    else:
      y.append(1)
  return y
  
y_train = label2int(train_original)
y_dev = label2int(dev_original)
y_test = label2int(test_original)
y_train = np.array(y_train)
y_dev = np.array(y_dev)
y_test = np.array(y_test)

print(y_train[0])
print(y_train[1])
print(y_train[2])
print(y_train[3])
print(y_train[4])

0
0
0
0
1


We also generate these target labels using one hot encoding.

In [10]:
def int2onehot(dataset):
  y = []
  for example in dataset:
    if example:
      y.append(np.array([0,1]))
    else:
      y.append(np.array([1,0]))
  return np.array(y)
y_train_onehot = int2onehot(y_train)
y_dev_onehot = int2onehot(y_dev)
y_test_onehot = int2onehot(y_test)

print(y_train_onehot[0])
print(y_train_onehot[1])
print(y_train_onehot[2])
print(y_train_onehot[3])
print(y_train_onehot[4])

[1 0]
[1 0]
[1 0]
[1 0]
[0 1]


In [11]:
start = ["<SEP>"]
sep ,masks,segments = tokenize(start, tokenizer)
print(sep)

[[  101  1026 19802  1028   102     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0]]


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


he easiest way is to input the tweet and topic as paired sentences into the model, concatenating them and separating them by the special BERT sentence-separator token [SEP] Then we can apply BERT directly.

In [12]:
# Please write your code to combine the tweet and topic into the following varibles

x_train_combined_b4_tokenize = []
x_dev_combined_b4_tokenize = []
x_test_combined_b4_tokenize = []



sep = "[SEP]"
#sep ,masks,segments = tokenize(start, tokenizer)

# Tips: 
# 1) We can use the special token <SEP> to concatenate the tweets and topics.
# 2) After combine them, make sure they are paded.

# your code goes here
for w in range (len(x_dev_tweetb4tokenize)):                 # Combining the tweets and the topics in the train set
    x_dev_combined = x_dev_tweetb4tokenize[w] + sep + x_dev_topicb4tokenize[w]   # Adds [SEP] in between
    x_dev_combined_b4_tokenize.append(x_dev_combined)


for w in range (len(x_train_tweetb4tokenize)):                 # Combining the tweets and the topics in the train set
    x_train_combined = x_train_tweetb4tokenize[w] + sep + x_train_topicb4tokenize[w]   # Adds [SEP] in between
    x_train_combined_b4_tokenize.append(x_train_combined)

for w in range (len(x_test_tweetb4tokenize)):                 # Combining the tweets and the topics in the train set
    x_test_combined = x_test_tweetb4tokenize[w] + sep + x_test_topicb4tokenize[w]   # Adds [SEP] in between
    x_test_combined_b4_tokenize.append(x_test_combined)

x_train_int, x_train_masks, x_train_segments = tokenize(x_train_combined_b4_tokenize, tokenizer)  #tokenize combined sentences
x_dev_int, x_dev_masks, x_dev_segments = tokenize(x_dev_combined_b4_tokenize, tokenizer) 
x_test_int, x_test_masks, x_test_segments = tokenize(x_test_combined_b4_tokenize, tokenizer) 


# Don't forget the to use np.array function to wrap the output of pad_sequences function
x_train_int_np = np.array(x_train_int)
x_train_masks_np = np.array(x_train_masks)
x_dev_int_np = np.array(x_dev_int)
x_dev_masks_np = np.array(x_dev_masks)
x_test_int_np = np.array(x_test_int)
x_test_masks_np = np.array(x_test_masks)


print(x_dev_int[0])
print(x_dev_masks[0],'\n')
print(x_dev_int_np[0])
print(x_dev_masks_np[0])

print(x_train_combined_b4_tokenize[0:2])


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


[  101  6108  1062  9794 16021 23091  2007 16839  9080 12863  7050  2000
  2745  4027  1024  6108  1062  4593  2587 16021 23091  2006  5095  1998
  1012  1012  8299  1024  1013  1013  1056  1012  2522  1013  1053  3501
  2683  2072  2549  8586  2615 18037   102  2745  4027   102     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In your report, show a few examples of your input, with text and topic included but separated by [SEP].

## Model 1: Prebuilt Sequence Classification



The huggingface transformer package provides many prebuilt models. First, let us try a basic text classification model based on DistilBERT. This first method is the standard way BERT models are used for text classification: we use the embedding of the special [CLS] token at the beginning of the sequence, and put it through a simple classifier layer to make the decision. The TFDistilBertForSequenceClassification class takes care of that for us.

The models with BERT are much bigger than our previous models. To run it faster, we can use TPU here. The detailed guideline about using TPU can be found from https://www.tensorflow.org/guide/tpu.

In [13]:
from transformers import TFDistilBertForSequenceClassification, DistilBertConfig
import tensorflow as tf

distil_bert = 'distilbert-base-uncased'

config = DistilBertConfig(num_labels=2)
config.output_hidden_states = False

def create_TFDistilBertForSequenceClassification():
    transformer_model = TFDistilBertForSequenceClassification.from_pretrained(distil_bert, config = config)
    input_ids = tf.keras.layers.Input(shape=(128,), name='input_token', dtype='int32')
    input_masks_ids = tf.keras.layers.Input(shape=(128,), name='masked_token', dtype='int32')
    X = transformer_model(input_ids, input_masks_ids)
    return tf.keras.Model(inputs=[input_ids, input_masks_ids], outputs = X)

use_tpu = True
if use_tpu:
  # Create distribution strategy
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)

  # Create model on TPU:
    with strategy.scope():
        model = create_TFDistilBertForSequenceClassification()
        optimizer = keras.optimizers.Adam(lr=5e-5)
        model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
else:
    model = create_TFDistilBertForSequenceClassification()
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])


INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Initializing the TPU system: grpc://10.78.16.18:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.78.16.18:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [14]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_token (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
masked_token (InputLayer)       [(None, 128)]        0                                            
__________________________________________________________________________________________________
tf_distil_bert_for_sequence_cla TFSequenceClassifier 66955010    input_token[0][0]                
                                                                 masked_token[0][0]               
Total params: 66,955,010
Trainable params: 66,955,010
Non-trainable params: 0
__________________________________________________________________________________________________


In [15]:
history = model.fit([x_train_int_np,x_train_masks_np],
                    y_train_onehot,
                    epochs=30,
                    batch_size=512,
                    validation_data=([x_dev_int_np,x_dev_masks_np], y_dev_onehot),
                    verbose=1)

Epoch 1/30


35/35 [==============================] - ETA: 0s - loss: 0.6505 - accuracy: 0.7890WARNING:tensorflow:The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


35/35 [==============================] - 90s 1s/step - loss: 0.6462 - accuracy: 0.7898 - val_loss: 0.4516 - val_accuracy: 0.7683
Epoch 2/30
35/35 [==============================] - 7s 206ms/step - loss: 0.3472 - accuracy: 0.8662 - val_loss: 0.3522 - val_accuracy: 0.8438
Epoch 3/30
35/35 [==============================] - 7s 206ms/step - loss: 0.2500 - accuracy: 0.9137 - val_loss: 0.4642 - val_accuracy: 0.8423
Epoch 4/30
35/35 [==============================] - 7s 204ms/step - loss: 0.2398 - accuracy: 0.9182 - val_loss: 0.4234 - val_accuracy: 0.8513
Epoch 5/30
35/35 [==============================] - 7s 208ms/step - loss: 0.1800 - accuracy: 0.9431 - val_loss: 0.4448 - val_accuracy: 0.8317
Epoch 6/30
35/35 [==============================] - 7s 206ms/step - loss: 0.2212 - accuracy: 0.9216 - val_loss: 0.6452 - val_accuracy: 0.8453
Epoch 7/30
35/35 [==============================] - 7s 207ms/step - loss: 0.1330 - accuracy: 0.9626 - val_loss: 0.7597 - val_accuracy: 0.8506
Epoch 8/30
35/35 [=

In [16]:
results = model.evaluate([x_test_int_np,x_test_masks_np], y_test_onehot)
print(results)

194/194 [==============================] - 8s 31ms/step - loss: 1.4850 - accuracy: 0.8627
[1.484992504119873, 0.8627324104309082]



## Model 2: Neural bag of words using BERT

In this model, we take the NBOW classifier from lab 4 (model3-1) and integrate BERT. Instead of averaging over word2vec or GloVe word vectors, we are averaging over the embedding representations produced by BERT - but otherwise, the classifier is the same.


In [17]:
class GlobalAveragePooling1DMasked(GlobalAveragePooling1D):
    def call(self, x, mask=None):
        if mask != None:
            return K.sum(x, axis=1) / K.sum(mask, axis=1)
        else:
            return super().call(x)

In [18]:
from transformers import TFDistilBertModel, DistilBertConfig

def get_BERT_layer():
  distil_bert = 'distilbert-base-uncased'
  config = DistilBertConfig(dropout=0.2, attention_dropout=0.2)
  config.output_hidden_states = False
  return TFDistilBertModel.from_pretrained(distil_bert, config = config)


In [19]:

hdepth=16
MAX_SEQUENCE_LENGTH = 128
EMBED_SIZE=100


def create_bag_of_words_BERT():
  input_ids_in = tf.keras.layers.Input(shape=(128,), name='input_token', dtype='int32')
  input_masks_in = tf.keras.layers.Input(shape=(128,), name='masked_token', dtype='int32') 

  bert_embeddings = get_BERT_layer()
  embedded_sent = bert_embeddings(input_ids_in, attention_mask=input_masks_in)[0]

  pooled_sent=GlobalAveragePooling1DMasked()(embedded_sent)
  hidden_output=Dense(hdepth,input_shape=(MAX_SEQUENCE_LENGTH,EMBED_SIZE),activation='sigmoid',kernel_initializer='glorot_uniform')(pooled_sent) # Sigmoid
  label=Dense(1,input_shape=(hdepth,),activation='sigmoid',kernel_initializer='glorot_uniform')(hidden_output)
  return Model(inputs=[input_ids_in,input_masks_in], outputs=[label],name='Model2_BERT')

use_tpu = True
if use_tpu:
  # Create distribution strategy
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.experimental.TPUStrategy(tpu)

  # Create model
  with strategy.scope():
    model2 = create_bag_of_words_BERT()
    optimizer2 = keras.optimizers.Adam(lr=5e-5)
    model2.compile(optimizer=optimizer2, loss='binary_crossentropy', metrics=['accuracy'])
else:
  model2 = create_bag_of_words_BERT()
  model2.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model2.summary() 




INFO:tensorflow:Initializing the TPU system: grpc://10.78.16.18:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.78.16.18:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)
Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['activation_13', 'vocab_layer_norm', 'vocab_projector', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertMo

Model: "Model2_BERT"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_token (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
masked_token (InputLayer)       [(None, 128)]        0                                            
__________________________________________________________________________________________________
tf_distil_bert_model (TFDistilB TFBaseModelOutput(la 66362880    input_token[0][0]                
                                                                 masked_token[0][0]               
__________________________________________________________________________________________________
global_average_pooling1d_masked (None, 768)          0           tf_distil_bert_model[0]

In [20]:

history = model2.fit([x_train_int_np,x_train_masks_np],
                    y_train,
                    epochs=30,
                    batch_size=512,
                    validation_data=([x_dev_int_np,x_dev_masks_np], y_dev),
                    verbose=1)

Epoch 1/30


35/35 [==============================] - ETA: 0s - loss: 0.4270 - accuracy: 0.8171WARNING:tensorflow:The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


35/35 [==============================] - 93s 1s/step - loss: 0.4253 - accuracy: 0.8179 - val_loss: 0.3329 - val_accuracy: 0.8468
Epoch 2/30
35/35 [==============================] - 8s 222ms/step - loss: 0.2449 - accuracy: 0.9045 - val_loss: 0.3313 - val_accuracy: 0.8430
Epoch 3/30
35/35 [==============================] - 7s 203ms/step - loss: 0.2036 - accuracy: 0.9285 - val_loss: 0.3418 - val_accuracy: 0.8506
Epoch 4/30
35/35 [==============================] - 7s 207ms/step - loss: 0.1680 - accuracy: 0.9492 - val_loss: 0.3443 - val_accuracy: 0.8475
Epoch 5/30
35/35 [==============================] - 7s 206ms/step - loss: 0.1527 - accuracy: 0.9575 - val_loss: 0.3617 - val_accuracy: 0.8325
Epoch 6/30
35/35 [==============================] - 7s 207ms/step - loss: 0.1276 - accuracy: 0.9704 - val_loss: 0.3824 - val_accuracy: 0.8445
Epoch 7/30
35/35 [==============================] - 7s 206ms/step - loss: 0.1152 - accuracy: 0.9764 - val_loss: 0.4008 - val_accuracy: 0.8234
Epoch 8/30
35/35 [=

In [21]:
results = model2.evaluate([x_test_int_np,x_test_masks_np], y_test)
print(results)

194/194 [==============================] - 8s 30ms/step - loss: 0.4000 - accuracy: 0.8862
[0.39996054768562317, 0.8861762285232544]


## Model 3: CNN or LSTM with BERT

In [ ]:
results = modelcnn.evaluate([x_test_int_np,x_test_masks_np], y_test)
print(results)

In [33]:
from keras.layers import Input, Embedding, Dense, LSTM
from keras.models import Model
from keras.models import Sequential

EMBED_SIZE = 100

#Diff style
def create_LSTM():
  input_ids_in = tf.keras.layers.Input(shape=(128,), name='input_token', dtype='int32')
  input_masks_in = tf.keras.layers.Input(shape=(128,), name='masked_token', dtype='int32') 

  bert_embeddings = get_BERT_layer()
  embedded_sent = bert_embeddings(input_ids_in, attention_mask=input_masks_in)[0]

  lstm = LSTM(100, return_sequences=False)(embedded_sent)
  DenseOutput = Dense(1, activation='softmax')(lstm)
  return Model(inputs=[input_ids_in,input_masks_in], outputs=[DenseOutput])  

use_tpu = True
if use_tpu:
  # Create distribution strategy
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.experimental.TPUStrategy(tpu)

  # Create model
  with strategy.scope():
    modellstm = create_LSTM()
    optimizer2 = keras.optimizers.Adam(lr=5e-5)
    modellstm.compile(optimizer=optimizer2, loss='binary_crossentropy', metrics=['accuracy'])

else:
  modellstm = create_LSTM()
  modellstm.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
modellstm.summary()


INFO:tensorflow:Initializing the TPU system: grpc://10.78.16.18:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.78.16.18:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)
Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['activation_13', 'vocab_layer_norm', 'vocab_projector', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertMo

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_token (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
masked_token (InputLayer)       [(None, 128)]        0                                            
__________________________________________________________________________________________________
tf_distil_bert_model_9 (TFDisti TFBaseModelOutput(la 66362880    input_token[0][0]                
                                                                 masked_token[0][0]               
__________________________________________________________________________________________________
lstm_5 (LSTM)                   (None, 100)          347600      tf_distil_bert_model_9[0][0

In [34]:
history = modellstm.fit([x_train_int_np,x_train_masks_np],
                    y_train,
                    epochs=30,
                    batch_size=512,
                    validation_data=([x_dev_int_np,x_dev_masks_np], y_dev),
                    verbose=1)

Epoch 1/30


35/35 [==============================] - ETA: 0s - loss: 0.4642 - accuracy: 0.7894WARNING:tensorflow:The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


35/35 [==============================] - 97s 1s/step - loss: 0.4618 - accuracy: 0.7895 - val_loss: 0.3378 - val_accuracy: 0.7442
Epoch 2/30
35/35 [==============================] - 10s 280ms/step - loss: 0.2302 - accuracy: 0.7967 - val_loss: 0.3138 - val_accuracy: 0.7442
Epoch 3/30
35/35 [==============================] - 7s 212ms/step - loss: 0.1646 - accuracy: 0.7876 - val_loss: 0.3186 - val_accuracy: 0.7442
Epoch 4/30
35/35 [==============================] - 7s 213ms/step - loss: 0.1201 - accuracy: 0.7886 - val_loss: 0.3672 - val_accuracy: 0.7442
Epoch 5/30
35/35 [==============================] - 7s 214ms/step - loss: 0.0843 - accuracy: 0.7929 - val_loss: 0.3881 - val_accuracy: 0.7442
Epoch 6/30
35/35 [==============================] - 7s 214ms/step - loss: 0.0595 - accuracy: 0.7914 - val_loss: 0.4347 - val_accuracy: 0.7442
Epoch 7/30
35/35 [==============================] - 7s 212ms/step - loss: 0.0412 - accuracy: 0.7893 - val_loss: 0.5607 - val_accuracy: 0.7442
Epoch 8/30
35/35 [

In [36]:
results = modellstm.evaluate([x_test_int_np,x_test_masks_np], y_test)
print(results)

194/194 [==============================] - 5s 24ms/step - loss: 0.6879 - accuracy: 0.3982
[0.6879350543022156, 0.398221492767334]
